# Notebook 05: Identity & OAuth Integration

## Learning Objectives
- Set up Google Drive OAuth integration with AgentCore Identity
- Implement secure credential management with session binding
- Deploy travel agent with OAuth2 authentication
- Create identity-aware travel tools for Google Drive
- Test complete OAuth2 3-legged flow

## Prerequisites
- Completed Notebook 03 (Gateway Integration) - Required for Cognito pool
- Completed Notebook 04 (Memory Implementation)
- Google Cloud Console account
- Google Drive API enabled
- OAuth 2.0 credentials configured
- Docker running

## Step 1: Connect to your AWS environment

In [ ]:
import os

os.environ['AWS_REGION'] = 'us-east-1'

# APPROACH A: Use credentials
# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_key'
# os.environ['AWS_SESSION_TOKEN'] = "your_session_token"

# APPROACH B: Use AWS SSO profile
#os.environ['AWS_PROFILE'] = 'your_profile'
# Remove any existing credential env vars to force profile usage
#for key in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN']:
#    os.environ.pop(key, None)

os.environ['AWS_REGION'] = 'us-east-1'

print("✅ AWS Profile set. Please restart kernel and run all cells.")

In [ ]:
import os

import subprocess
from boto3.session import Session
from bedrock_agentcore.services.identity import IdentityClient
from bedrock_agentcore.identity.auth import requires_access_token
from bedrock_agentcore_starter_toolkit import Runtime

# Import required modules for Identity setup
print("✅ AgentCore Identity imports ready")

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")
print("✅ AgentCore Identity imports successful")

## Step 2: Create Test User in Existing Cognito Pool

In [ ]:
# Load existing Cognito config from notebook 03
import sys
import boto3

sys.path.append('../backend')

from cognito_config import load_cognito_config, ensure_user_password_auth

GATEWAY_NAME = "TravelMateGateway"

print("🔐 Loading existing Cognito configuration...")
cognito_result = load_cognito_config(GATEWAY_NAME)

if not cognito_result:
    print("❌ No existing Cognito config found. Please run notebook 03 first.")
    raise Exception("Missing Cognito configuration from notebook 03")

# Create test user in existing pool
print("👤 Creating test user in existing Cognito pool...")
cognito_client = boto3.client('cognito-idp', region_name=region)

# Get pool ID directly from client_info
pool_id = cognito_result['client_info']['user_pool_id']

try:
    # Create test user
    cognito_client.admin_create_user(
        UserPoolId=pool_id,
        Username='testuser',
        TemporaryPassword='Temp123!',
        MessageAction='SUPPRESS'
    )
    
    # Set permanent password
    cognito_client.admin_set_user_password(
        UserPoolId=pool_id,
        Username='testuser',
        Password='MyPassword123!',
        Permanent=True
    )
    print("✅ Test user created: testuser / MyPassword123!")
except cognito_client.exceptions.UsernameExistsException:
    print("✅ Test user already exists: testuser / MyPassword123!")

# Ensure USER_PASSWORD_AUTH is enabled
ensure_user_password_auth(
    cognito_result['client_info']['client_id'],
    cognito_result['client_info']['user_pool_id'],
    region
)

print("✅ Cognito configuration loaded")
print(f"Client ID: {cognito_result['client_info']['client_id']}")
print(f"Scope: {cognito_result['client_info']['scope']}")

## Step 3: Google OAuth Setup Guide

### Configure Google Drive API Access

Follow these steps to set up Google Drive integration:

#### 1. Create Google Cloud Project
- Go to [Google Cloud Console](https://console.cloud.google.com/)
- Create a new project or select existing one

#### 2. Enable Google Drive API
- Navigate to **APIs & Services > Library**
- Search for "Google Drive API"
- Click **Enable**

#### 3. Configure OAuth Consent Screen
- Go to **APIs & Services > OAuth consent screen**
- Choose **External** user type
- Fill required fields:
  - App name: "AI Travel Companion"
  - User support email: Your email
  - Developer contact: Your email
- Add your email as a test user

#### 4. Create OAuth 2.0 Credentials
- Go to **APIs & Services > Credentials**
- Click **Create Credentials > OAuth client ID**
- Choose **Web application**
- Name: "Travel Companion OAuth"
- **Important**: We'll add the callback URL after creating the provider

#### 5. Get Client ID and Secret
- Copy the **Client ID** and **Client Secret**

In [ ]:
# Google OAuth credentials
os.environ['GOOGLE_CLIENT_ID'] = ""
os.environ['GOOGLE_CLIENT_SECRET'] = ""

google_client_id = os.getenv("GOOGLE_CLIENT_ID")
google_client_secret = os.getenv("GOOGLE_CLIENT_SECRET")

print(f"✅ Google OAuth credentials configured")
print(f"Client ID: {google_client_id[:20]}...")

## Step 4: Create OAuth2 Credential Provider

In [ ]:
# Configuration
PROVIDER_NAME = "google-drive-provider"
identity_client = IdentityClient(region)

print("🔐 Creating Google Drive OAuth2 Credential Provider...")

# Since the provider was deleted, create a new one
try:
    google_provider = identity_client.create_oauth2_credential_provider({
        "name": PROVIDER_NAME,
        "credentialProviderVendor": "GoogleOauth2",
        "oauth2ProviderConfigInput": {
            "googleOauth2ProviderConfig": {
                "clientId": google_client_id,
                "clientSecret": google_client_secret
            }
        }
    })
    print(f"✅ Created new OAuth2 Provider: {google_provider['name']}")
    print(f"📋 AgentCore Callback URL: {google_provider['callbackUrl']}")
    print("\n⚠️ IMPORTANT: Add this callback URL to your Google OAuth2 client configuration!")
except Exception as e:
    print(f"❌ Error creating OAuth2 provider: {str(e)}")
    print("\n💡 This might be due to a temporary SecretsManager issue. Please try again in a few minutes.")

## Step 5: Create OAuth2 Callback Server

In [ ]:
%%writefile ../backend/identity/runtime/oauth2_callback_server.py
#!/usr/bin/env python3
"""
OAuth2 Callback Server for Google Drive Integration
Handles OAuth2 3-legged authentication flow with AgentCore Identity
"""

import time
import uvicorn
import logging
import argparse
import requests

from datetime import timedelta
from fastapi import FastAPI, HTTPException, status
from fastapi.responses import HTMLResponse
from bedrock_agentcore.services.identity import IdentityClient, UserTokenIdentifier

# Configuration constants
OAUTH2_CALLBACK_SERVER_PORT = 9090
PING_ENDPOINT = "/ping"
OAUTH2_CALLBACK_ENDPOINT = "/oauth2/callback"
USER_IDENTIFIER_ENDPOINT = "/userIdentifier/token"

logger = logging.getLogger(__name__)

class OAuth2CallbackServer:
    def __init__(self, region: str):
        self.identity_client = IdentityClient(region=region)
        self.user_token_identifier = None
        self.app = FastAPI()
        self._setup_routes()

    def _setup_routes(self):
        @self.app.post(USER_IDENTIFIER_ENDPOINT)
        async def _store_user_token(user_token_identifier_value: UserTokenIdentifier):
            self.user_token_identifier = user_token_identifier_value

        @self.app.get(PING_ENDPOINT)
        async def _handle_ping():
            return {"status": "success"}

        @self.app.get(OAUTH2_CALLBACK_ENDPOINT)
        async def _handle_oauth2_callback(session_id: str):
            if not session_id:
                raise HTTPException(
                    status_code=status.HTTP_400_BAD_REQUEST,
                    detail="Missing session_id query parameter",
                )

            if not self.user_token_identifier:
                logger.error("No configured user token identifier")
                raise HTTPException(
                    status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
                    detail="Internal Server Error",
                )

            self.identity_client.complete_resource_token_auth(
                session_uri=session_id, user_identifier=self.user_token_identifier
            )

            html_content = """
            <!DOCTYPE html>
            <html>
            <head>
                <title>OAuth2 Success</title>
                <style>
                    body {
                        margin: 0; padding: 0; height: 100vh;
                        display: flex; justify-content: center; align-items: center;
                        font-family: Arial, sans-serif; background-color: #f5f5f5;
                    }
                    .container {
                        text-align: center; padding: 2rem; background-color: white;
                        border-radius: 8px; box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
                    }
                    h1 { color: #28a745; margin: 0; }
                </style>
            </head>
            <body>
                <div class="container">
                    <h1>✅ Google Drive OAuth2 Authorization Successful!</h1>
                    <p>You can now close this window and return to the application.</p>
                </div>
            </body>
            </html>
            """
            return HTMLResponse(content=html_content, status_code=200)

    def get_app(self) -> FastAPI:
        return self.app

def get_oauth2_callback_url() -> str:
    return f"http://localhost:{OAUTH2_CALLBACK_SERVER_PORT}{OAUTH2_CALLBACK_ENDPOINT}"

def store_token_in_oauth2_callback_server(user_token_value: str):
    if user_token_value:
        requests.post(
            f"http://localhost:{OAUTH2_CALLBACK_SERVER_PORT}{USER_IDENTIFIER_ENDPOINT}",
            json={"user_token": user_token_value},
            timeout=2,
        )

def wait_for_oauth2_server_to_be_ready(duration: timedelta = timedelta(seconds=40)) -> bool:
    timeout_in_seconds = duration.seconds
    start_time = time.time()
    
    while time.time() - start_time < timeout_in_seconds:
        try:
            response = requests.get(
                f"http://localhost:{OAUTH2_CALLBACK_SERVER_PORT}{PING_ENDPOINT}",
                timeout=2,
            )
            if response.status_code == status.HTTP_200_OK:
                return True
        except requests.exceptions.RequestException:
            pass
        time.sleep(2)
    
    return False

def main():
    parser = argparse.ArgumentParser(description="OAuth2 Callback Server")
    parser.add_argument("-r", "--region", type=str, required=True, help="AWS Region")
    args = parser.parse_args()
    
    oauth2_callback_server = OAuth2CallbackServer(region=args.region)
    uvicorn.run(
        oauth2_callback_server.get_app(),
        host="127.0.0.1",
        port=OAUTH2_CALLBACK_SERVER_PORT,
    )

if __name__ == "__main__":
    main()

## Step 6: Create Travel Agent with Google Drive Integration

In [ ]:
%%writefile ../backend/identity/runtime/travel_agent_google_drive.py
#!/usr/bin/env python3
"""
Travel Agent with Google Drive Integration
Uses AgentCore Identity for OAuth2 authentication with Google Drive
"""

import os
import json
import asyncio
import io
from datetime import datetime
from typing import Dict, Any, Optional, AsyncGenerator

from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token
from oauth2_callback_server import get_oauth2_callback_url

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from googleapiclient.errors import HttpError

# Environment configuration
os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

# Google Drive API scope
SCOPES = ["https://www.googleapis.com/auth/drive.file"]

# Global variable to store access token
google_access_token: Optional[str] = None

@tool(
    name="save_itinerary_to_drive",
    description="Saves a travel itinerary to Google Drive as a text file"
)
def save_itinerary_to_drive(destination: str, itinerary_content: str) -> str:
    """
    Save travel itinerary to Google Drive.
    
    Args:
        destination: Travel destination name
        itinerary_content: The itinerary content to save
    
    Returns:
        str: Success message with file link or error message
    """
    global google_access_token
    
    if not google_access_token:
        return json.dumps({
            "message": "Google Drive authentication is required. Please wait while we set up the authorization.",
            "success": False
        })
    
    try:
        # Create credentials from access token
        creds = Credentials(token=google_access_token, scopes=SCOPES)
        service = build('drive', 'v3', credentials=creds)
        
        # Create filename
        filename = f"{destination.lower().replace(' ', '_')}_itinerary_{datetime.now().strftime('%Y%m%d')}.txt"
        
        # Create file metadata
        file_metadata = {'name': filename}
        
        # Create media upload
        media = MediaIoBaseUpload(
            io.BytesIO(itinerary_content.encode('utf-8')),
            mimetype='text/plain'
        )
        
        # Upload file
        file = service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id,name,webViewLink'
        ).execute()
        
        return json.dumps({
            "success": True,
            "message": f"✅ Itinerary saved to Google Drive: {file.get('name')}",
            "file_id": file.get('id'),
            "view_link": file.get('webViewLink')
        })
        
    except HttpError as error:
        return json.dumps({
            "success": False,
            "error": f"Google Drive API error: {str(error)}"
        })
    except Exception as e:
        return json.dumps({
            "success": False,
            "error": f"Error saving to Google Drive: {str(e)}"
        })

# Initialize the agent
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[save_itinerary_to_drive],
    system_prompt="""
You are a helpful travel planning assistant with the ability to save itineraries to Google Drive.
When users ask you to create travel plans, generate detailed itineraries and offer to save them to Google Drive.
Always format itineraries clearly with day-by-day breakdowns, times, and activities.
"""
)

# Initialize app
app = BedrockAgentCoreApp()

class StreamingQueue:
    def __init__(self):
        self.finished = False
        self.queue = asyncio.Queue()
        
    async def put(self, item: str) -> None:
        await self.queue.put(item)

    async def finish(self) -> None:
        self.finished = True
        await self.queue.put(None)

    async def stream(self) -> AsyncGenerator[str, None]:
        while True:
            item = await self.queue.get()
            if item is None and self.finished:
                break
            yield item

queue = StreamingQueue()

async def on_auth_url(url: str) -> None:
    """Handle authorization URL callback."""
    print(f"Authorization url: {url}")
    await queue.put(f"Authorization url: {url}")

async def agent_task(user_message: str) -> None:
    """Execute the agent task with authentication handling."""
    try:
        await queue.put("Begin agent execution")
        
        # Call the agent first to see if it needs authentication
        response = agent(user_message)
        
        # Extract text content from the response structure
        response_text = ""
        if isinstance(response.message, dict):
            content = response.message.get('content', [])
            if isinstance(content, list):
                for item in content:
                    if isinstance(item, dict) and 'text' in item:
                        response_text += item['text']
        else:
            response_text = str(response.message)
        
        # Check if the response indicates authentication is required
        auth_keywords = [
            "authentication", "authorize", "authorization", "auth", 
            "sign in", "login", "access", "permission", "credential",
            "need authentication", "requires authentication"
        ]
        needs_auth = any(keyword.lower() in response_text.lower() for keyword in auth_keywords)
       
        if needs_auth:
            await queue.put("Authentication required for Google Drive access. Starting authorization flow...")
            
            # Trigger the 3LO authentication flow
            try:
                global google_access_token
                google_access_token = await get_google_drive_token(access_token='')
                await queue.put("Authentication successful! Retrying your request...")
                
                # Retry the agent call now that we have authentication
                response = agent(user_message)
            except Exception as auth_error:
                print(f"auth_error: {auth_error}")
                await queue.put(f"Authentication failed: {str(auth_error)}")
        
        await queue.put(response.message)
        await queue.put("End agent execution")
    except Exception as e:
        await queue.put(f"Error: {str(e)}")
    finally:
        await queue.finish()

@requires_access_token(
    provider_name="google-drive-provider",
    scopes=SCOPES,
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=True,
    callback_url=get_oauth2_callback_url()
)
async def get_google_drive_token(*, access_token: str) -> str:
    """Get Google Drive access token."""
    global google_access_token
    google_access_token = access_token
    return access_token

@app.entrypoint
async def agent_invocation(payload: Dict[str, Any]) -> AsyncGenerator[str, None]:
    """Main entrypoint for agent invocations."""
    user_message = payload.get(
        "prompt", 
        "Hello! I'm your travel planning assistant. How can I help you plan your next trip?"
    )
    
    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))
    
    # Stream results
    async def stream_with_task() -> AsyncGenerator[str, None]:
        async for item in queue.stream():
            yield item
        await task
    
    return stream_with_task()

if __name__ == "__main__":
    app.run()

## Step 7: Configure AgentCore Runtime Deployment

In [ ]:
%%writefile ../backend/identity/runtime/requirements.txt
bedrock-agentcore
strands-agents
google-api-python-client
google-auth-httplib2
google-auth-oauthlib
fastapi
uvicorn
requests

In [ ]:
print("🚀 Configuring AgentCore Runtime deployment...")

# Change to backend directory
backend_dir = os.path.abspath('../backend/identity/runtime')
os.makedirs(backend_dir, exist_ok=True)
original_dir = os.getcwd()
os.chdir(backend_dir)

try:
    discovery_url = cognito_result['authorizer_config']['customJWTAuthorizer']['discoveryUrl']
    client_id = cognito_result['client_info']['client_id']

    agentcore_runtime = Runtime()
    
    response = agentcore_runtime.configure(
        entrypoint="travel_agent_google_drive.py",
        auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        memory_mode="NO_MEMORY",
        agent_name="travel_agent_google_drive",
        authorizer_configuration={
            "customJWTAuthorizer": {
                "discoveryUrl": discovery_url,
                "allowedClients": [client_id]
            }
        } if discovery_url and client_id else None
    )
    
    print("✅ Runtime configuration completed")
    print(response)
finally:
    print("Creation of Runtime configuration FINISHED.")

## Step 8: Start OAuth2 Callback Server

Before testing the OAuth2 flow, you need to start the local OAuth2 callback server that handles the 3-legged authentication flow.

### Running the OAuth2 Callback Server

Open a **separate terminal** and run the following command from the project root:

#### Option A
```bash
AWS_PROFILE=YOUR_PROFILE AWS_REGION=us-east-1 uv run python capstone_project/backend/identity/runtime/oauth2_callback_server.py --region us-east-1
```
#### Option B
```bash
export AWS_ACCESS_KEY_ID="YOUR_ACCESS_KEY"
export AWS_SECRET_ACCESS_KEY="YOUR_SECRET_KEY"
AWS_REGION=us-east-1 uv run python capstone_project/backend/identity/runtime/oauth2_callback_server.py --region us-east-1
```

**Important Notes:**
- Replace `YOUR_PROFILE` with your actual AWS profile name
- The server runs on `localhost:9090` and must stay running during OAuth testing
- The server needs AWS credentials to communicate with AgentCore Identity service
- You'll see output like: `INFO: Uvicorn running on http://127.0.0.1:9090`

**Keep this server running** while testing the OAuth2 flow in the next steps.

In [ ]:
print("💡 OAuth2 callback server setup instructions:")
print("")
print("1. Open a separate terminal")
print("2. Navigate to the project root directory")
print("3. Run the following command:")
print("")
print("   AWS_PROFILE=YOUR_PROFILE AWS_REGION=us-east-1 uv run python capstone_project/backend/identity/runtime/oauth2_callback_server.py --region us-east-1")
print("")
print("⚠️ Replace 'YOUR_PROFILE' with your actual AWS profile name")
print("⚠️ Keep this server running during OAuth2 testing!")
print("⚠️ The server must have AWS credentials to communicate with AgentCore Identity")
print("")
print("✅ You should see: 'INFO: Uvicorn running on http://127.0.0.1:9090'")
print("✅ The server handles OAuth callbacks and session binding")
print("✅ Leave the server running and proceed to the next step")

## Step 9: Deploy Agent to AgentCore Runtime

In [ ]:
from oauth2_callback_server import get_oauth2_callback_url

print("🚀 Deploying agent to AgentCore Runtime...")

# Deploy the agent
launch_result = agentcore_runtime.launch()
print(f"✅ Agent deployed: {launch_result.agent_id}")


In [ ]:
if launch_result.agent_id:
    # Update workload identity with OAuth2 callback URL
    workload_name = launch_result.agent_id
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    allowed_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    oauth2_callback_url = get_oauth2_callback_url()
    
    print(f"🔗 Updating workload {workload_name} with callback URL: {oauth2_callback_url}")
    
    # Register the callback URL
    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_urls, oauth2_callback_url],
    )
    
    print("✅ Workload identity updated with OAuth2 callback URL")
else:
    print("❌ Failed to get agent ID from deployment")

os.chdir(original_dir)

## Step 10: Verify Deployment Status

In [ ]:
import time

print("⏳ Waiting for AgentCore Runtime to be ready...")

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    print(f"Status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

print(f"✅ Final status: {status}")

if status == 'READY':
    print("🎉 Agent is ready for testing!")
else:
    print(f"❌ Deployment failed with status: {status}")

## Step 11: Test the Travel Agent with Google Drive Integration

In [ ]:
import sys

#sys.path.append('../../')

from auth_utils import reauthenticate_user

print("🧪 Testing Travel Agent with Google Drive Integration...")

# Get bearer token for authentication using test user
bearer_token = reauthenticate_user(cognito_result['client_info']['client_id'])

if bearer_token:
    print(f"✅ Got access token: {bearer_token[:20]}...")
    
    # Test the agent
    test_prompt = """
    Create a 3-day travel itinerary for Tokyo, Japan including:
    - Traditional temples and gardens
    - Modern attractions like Tokyo Skytree
    - Food experiences and restaurants
    - Shopping districts
    
    Please save this itinerary to my Google Drive without asking again if you're allow to.
    """
    
    print("🤖 Invoking travel agent...")
    try:
        from oauth2_callback_server import store_token_in_oauth2_callback_server
        store_token_in_oauth2_callback_server(bearer_token)

        invoke_response = agentcore_runtime.invoke(
            {"prompt": test_prompt},
            bearer_token=bearer_token
        )
        
        print("📝 Agent Response:")
        print(invoke_response)
        
    except Exception as e:
        print(f"❌ Error during testing: {str(e)}")
        print("\n💡 Note: To test OAuth2 flow, start the callback server manually:")
        print("   uv run python ../backend/identity/runtime/oauth2_callback_server.py --region us-east-1")
else:
    print("❌ Failed to get access token")

In [ ]:
# Add this cell to save identity configuration
import json
import os

# Save identity configuration for notebook 08
identity_config = {
    "oauth2_provider": {
        "name": PROVIDER_NAME,
        "provider_id": google_provider.get('credentialProviderId'),
        "callback_url": google_provider.get('callbackUrl'),
        "vendor": "GoogleOauth2"
    },
    "google_oauth": {
        "client_id": google_client_id,
    },
    "agent_runtime": {
        "name": "travel_agent_google_drive",
        "entrypoint": "travel_agent_google_drive.py",
        "oauth_callback_server_port": 9090
    },
    "cognito_integration": {
        "user_pool_id": cognito_result['client_info']['user_pool_id'],
        "client_id": cognito_result['client_info']['client_id'],
        "discovery_url": cognito_result['authorizer_config']['customJWTAuthorizer']['discoveryUrl']
    },
    "region": region
}

# Save to environments directory
with open('environments/identity_info.json', 'w') as f:
    json.dump(identity_config, f, indent=2)

print("✅ Identity configuration saved to environments/identity_info.json")
print(f"📋 OAuth2 Provider: {PROVIDER_NAME}")
print(f"📋 Callback URL: {google_provider.get('callbackUrl')}")


## Step 12: Integration Summary

### What We've Accomplished

1. **Complete OAuth2 Flow**: Implemented proper 3-legged OAuth with session binding
2. **Cognito Integration**: Set up inbound authentication with Cognito
3. **AgentCore Runtime**: Deployed travel agent to runtime with proper authorization
4. **Google Drive Integration**: Created tools to save travel itineraries to Google Drive
5. **Session Binding**: Implemented secure OAuth2 session binding with callback server
6. **Workload Identity**: Properly configured workload identity with callback URLs

### Key Features

- **Secure Authentication**: Uses AgentCore Identity for OAuth2 management
- **Session Binding**: Prevents OAuth token hijacking with proper user validation
- **Travel Planning**: AI-powered travel itinerary generation
- **Google Drive Storage**: Automatic saving of itineraries to user's Google Drive
- **Production Ready**: Deployed to AgentCore Runtime with proper authorization

### Next Steps

- **Google Console**: Ensure the AgentCore callback URL is added to your Google OAuth2 client
- **Testing**: Use the test credentials (testuser / MyPassword123!) for Cognito authentication
- **Expansion**: Add more travel tools like flight booking, hotel reservations, etc.
- **Production**: Deploy with proper HTTPS endpoints for production use

### Usage Flow

1. User authenticates with Cognito
2. User requests travel planning with Google Drive save
3. Agent triggers OAuth2 flow for Google Drive access
4. User authorizes Google Drive access in browser
5. Agent saves itinerary to Google Drive
6. User receives confirmation with Google Drive link

The travel agent is now fully integrated with Google Drive using proper AgentCore Identity OAuth2 flow!